# Capstone Main Project

## Opening a Chinese restaurant in London

## Description

#### We want to analyse London as a potential location for a new Chinese restaurant.
#### To do this, we will look at all 33 London boroughs to ascertain which one offers the highest likelihood of success. The objective is to find one Borough which has all the right attributes for a new Chinese restaurant to achieve profitability.

#### The first step is to get all the information required on London boroughs. We will scrape a wikipedia page containing location data, along with additional information concerning the boroughs

#### We will clean the data and arrange it in a dataframe

#### Next, we will extract from Foursquare venue information on a large radius, for each borough

#### We will group the venues in a way that serves to assess the designated criteria for the analysis.

#### We will run cluster analysis and group the borougs by the criteria

#### Finally, we will pick the cluster that has the attributes we desire, and within that cluster we will pick the most desirable borough to open the restaurant

### Location Selection Criteria
#### We want to pick a location which meets the following criteria:
####   - has a significant number of restaurants out of all venues uncovered (likely to attract diners)
####   - has a low number of Chinese restaurants (market not saturated)
####   - the share of Chinese restaurants out of all restaurants is relatively low (good diversifier)
####   - has a significant number of bars/pubs (people tend to eat more unhealthy food (Chinese food is quite unhealthy) when consuming alcohol
####   - has a significant number of hotels (Chinese food is unhealthy, so it's not recommended to have it often - thus the number of repeat customers is small; tourists, on the other hand, tend to want to try new things, and areas with plenty of hotels guarantee a constant flow of tourists, who are more likely to be the major customer base of our Chinese restaurant)
####   - has a relatively low number of gyms (health-aware people are less likely to consume Chinese food regularly)
####   - we like areas which are densely populated and have offices (takeaway potential)

### First import necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# import web-scraping library BS4 and requests
from bs4 import BeautifulSoup
import requests

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Scrape website, store table data in text file, create dataframe with table contents and clean dataframe

In [2]:
# scrape website and store data in html format
link = 'https://en.wikipedia.org/wiki/List_of_London_boroughs'
page_response = requests.get(link, timeout=5)
page_content = BeautifulSoup(page_response.content, "html.parser")
page_content

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of London boroughs - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_London_boroughs","wgTitle":"List of London boroughs","wgCurRevisionId":865499060,"wgRevisionId":865499060,"wgArticleId":28092685,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from August 2015","Use British English from August 2015","Lists of coordinates","Geographic coordinate lists","Articles with Geo","London boroughs","Lists of places in London"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransform

In [3]:
# select only the table data (list of boroughs) and store in text format
# notice from the html file that table entries have the <tr> class, so we select using this criteria
table_data = []
for i in range(0, len(page_content.find_all("tr"))):
    table_entries = page_content.find_all("tr")[i].text
    table_data.append(table_entries)

table_data

['\nBorough\n\nInner\n\nStatus\n\nLocal authority\n\nPolitical control\n\nHeadquarters\n\nArea (sq mi)\n\nPopulation (2013 est)[1]\n\nCo-ordinates\n\n Nr. in map \n',
 '\nBarking and Dagenham [note 1]\n\n\n\n\n\nBarking and Dagenham London Borough Council\n\nLabour\n\nTown Hall, 1 Town Square\n\n13.93\n\n194,352\n\n51°33′39″N 0°09′21″E\ufeff / \ufeff51.5607°N 0.1557°E\ufeff / 51.5607; 0.1557\ufeff (Barking and Dagenham)\n\n25\n',
 '\nBarnet\n\n\n\n\n\nBarnet London Borough Council\n\nConservative\n\nNorth London Business Park, Oakleigh Road South\n\n33.49\n\n369,088\n\n51°37′31″N 0°09′06″W\ufeff / \ufeff51.6252°N 0.1517°W\ufeff / 51.6252; -0.1517\ufeff (Barnet)\n\n31\n',
 '\nBexley\n\n\n\n\n\nBexley London Borough Council\n\nConservative\n\nCivic Offices, 2 Watling Street\n\n23.38\n\n236,687\n\n51°27′18″N 0°09′02″E\ufeff / \ufeff51.4549°N 0.1505°E\ufeff / 51.4549; 0.1505\ufeff (Bexley)\n\n23\n',
 '\nBrent\n\n\n\n\n\nBrent London Borough Council\n\nLabour\n\nBrent Civic Centre, Engineer

##### Store table data into a dataframe and clean it up

In [9]:
# firstly create dataframe, removing last 5 rows which are in excess
raw_table = pd.DataFrame(table_data)
raw_table

,0
0,\nBorough\n\nInner\n\nStatus\n\nLocal authorit...
1,\nBarking and Dagenham [note 1]\n\n\n\n\n\nBar...
2,\nBarnet\n\n\n\n\n\nBarnet London Borough Coun...
3,\nBexley\n\n\n\n\n\nBexley London Borough Coun...
4,\nBrent\n\n\n\n\n\nBrent London Borough Counci...
5,\nBromley\n\n\n\n\n\nBromley London Borough Co...
6,\nCamden\n\nY\n\n\n\nCamden London Borough Cou...
7,\nCroydon\n\n\n\n\n\nCroydon London Borough Co...
8,\nEaling\n\n\n\n\n\nEaling London Borough Coun...
9,\nEnfield\n\n\n\n\n\nEnfield London Borough Co...


In [10]:
# secondly split string in each row using '\n' separator, removing excess columns
# assign first row as column names and remove it afterwards
split_table = pd.DataFrame(raw_table[0].str.split('\n',-1).tolist()).iloc[:,1:-1]
split_table.columns = split_table.iloc[0,:]
split_table = split_table.iloc[1:,:]
split_table

,Borough,,Inner,,Status,,Local authority,,Political control,,Headquarters,,Area (sq mi),,Population (2013 est)[1],,Co-ordinates,,Nr. in map,,None,None,None,None,None,None,None,None,None,None,None,None
1,Barking and Dagenham [note 1],,,,,,Barking and Dagenham London Borough Council,,Labour,,"Town Hall, 1 Town Square",,13.93,,"194,352",,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,,25,,None,None,None,None,None,None,None,None,None,None,None,None
2,Barnet,,,,,,Barnet London Borough Council,,Conservative,,"North London Business Park, Oakleigh Road South",,33.49,,"369,088",,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,,31,,None,None,None,None,None,None,None,None,None,None,None,None
3,Bexley,,,,,,Bexley London Borough Council,,Conservative,,"Civic Offices, 2 Watling Street",,23.38,,"236,687",,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,,23,,None,None,None,None,None,None,None,None,None,None,None,None
4,Brent,,,,,,Brent London Borough Council,,Labour,,"Brent Civic Centre, Engineers Way",,16.70,,"317,264",,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,,12,,None,None,None,None,None,None,None,None,None,None,None,None
5,Bromley,,,,,,Bromley London Borough Council,,Conservative,,"Civic Centre, Stockwell Close",,57.97,,"317,899",,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,,20,,None,None,None,None,None,None,None,None,None,None,None,None
6,Camden,,Y,,,,Camden London Borough Council,,Labour,,"Camden Town Hall, Judd Street",,8.40,,"229,719",,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ /...,,11,,None,None,None,None,None,None,None,None,None,None,None,None
7,Croydon,,,,,,Croydon London Borough Council,,Labour,,"Bernard Weatherill House, Mint Walk",,33.41,,"372,752",,51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ /...,,19,,None,None,None,None,None,None,None,None,None,None,None,None
8,Ealing,,,,,,Ealing London Borough Council,,Labour,,"Perceval House, 14-16 Uxbridge Road",,21.44,,"342,494",,51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ /...,,13,,None,None,None,None,None,None,None,None,None,None,None,None
9,Enfield,,,,,,Enfield London Borough Council,,Labour,,"Civic Centre, Silver Street",,31.74,,"320,524",,51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ /...,,30,,None,None,None,None,None,None,None,None,None,None,None,None
10,Greenwich [note 2],,Y [note 3],,Royal,,Greenwich London Borough Council,,Labour,,"Woolwich Town Hall, Wellington Street",,18.28,,"264,008",,51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E﻿ /...,,22,,None,None,None,None,None,None,None,None,None,None,None,None


In [19]:
# drop excess rows and columns from dataframe
processed_table = split_table
processed_table = processed_table.drop(processed_table.index[[32]]).iloc[:33,:17]
processed_table

,Borough,,Inner,,Status,,Local authority,,Political control,,Headquarters,,Area (sq mi),,Population (2013 est)[1],,Co-ordinates
1,Barking and Dagenham [note 1],,,,,,Barking and Dagenham London Borough Council,,Labour,,"Town Hall, 1 Town Square",,13.93,,"194,352",,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...
2,Barnet,,,,,,Barnet London Borough Council,,Conservative,,"North London Business Park, Oakleigh Road South",,33.49,,"369,088",,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...
3,Bexley,,,,,,Bexley London Borough Council,,Conservative,,"Civic Offices, 2 Watling Street",,23.38,,"236,687",,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...
4,Brent,,,,,,Brent London Borough Council,,Labour,,"Brent Civic Centre, Engineers Way",,16.70,,"317,264",,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...
5,Bromley,,,,,,Bromley London Borough Council,,Conservative,,"Civic Centre, Stockwell Close",,57.97,,"317,899",,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...
6,Camden,,Y,,,,Camden London Borough Council,,Labour,,"Camden Town Hall, Judd Street",,8.40,,"229,719",,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ /...
7,Croydon,,,,,,Croydon London Borough Council,,Labour,,"Bernard Weatherill House, Mint Walk",,33.41,,"372,752",,51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ /...
8,Ealing,,,,,,Ealing London Borough Council,,Labour,,"Perceval House, 14-16 Uxbridge Road",,21.44,,"342,494",,51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ /...
9,Enfield,,,,,,Enfield London Borough Council,,Labour,,"Civic Centre, Silver Street",,31.74,,"320,524",,51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ /...
10,Greenwich [note 2],,Y [note 3],,Royal,,Greenwich London Borough Council,,Labour,,"Woolwich Town Hall, Wellington Street",,18.28,,"264,008",,51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E﻿ /...


In [48]:
# create new dataframe with only the columns of interest
LDN_clean = processed_table[['Borough','Headquarters','Area (sq mi)','Population (2013 est)[1]','Political control','Co-ordinates']]
LDN_clean.reset_index(drop=True, inplace=True)
LDN_clean

,Borough,Headquarters,Area (sq mi),Population (2013 est)[1],Political control,Co-ordinates
0,Barking and Dagenham [note 1],"Town Hall, 1 Town Square",13.93,"194,352",Labour,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...
1,Barnet,"North London Business Park, Oakleigh Road South",33.49,"369,088",Conservative,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...
2,Bexley,"Civic Offices, 2 Watling Street",23.38,"236,687",Conservative,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...
3,Brent,"Brent Civic Centre, Engineers Way",16.70,"317,264",Labour,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...
4,Bromley,"Civic Centre, Stockwell Close",57.97,"317,899",Conservative,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...
5,Camden,"Camden Town Hall, Judd Street",8.40,"229,719",Labour,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ /...
6,Croydon,"Bernard Weatherill House, Mint Walk",33.41,"372,752",Labour,51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ /...
7,Ealing,"Perceval House, 14-16 Uxbridge Road",21.44,"342,494",Labour,51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ /...
8,Enfield,"Civic Centre, Silver Street",31.74,"320,524",Labour,51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ /...
9,Greenwich [note 2],"Woolwich Town Hall, Wellington Street",18.28,"264,008",Labour,51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E﻿ /...


In [78]:
# refine the geocoordinates column
LDN_geo = LDN_clean[['Borough','Co-ordinates']]
LDN_geo_DMS = pd.DataFrame(LDN_geo['Co-ordinates'].str.split('/',-1).tolist())
LDN_geo_DMS.columns = ['Latitude_DMS','Longitude_DMS','Decimals']
LDN_geo_dec = pd.DataFrame(LDN_geo_DMS['Decimals'].str.split('; ',-1).tolist())
LDN_geo_dec.columns = ['Latitude','Longitude_raw']
LDN_geo_dec_L = pd.DataFrame(LDN_geo_dec['Longitude_raw'].str.split(' ',-1).tolist())
LDN_geo_dec_L.columns = ['Longitude',1,2,3]
LDN_geo_dec_L['Longitude'] = LDN_geo_dec_L['Longitude'].map(lambda x: x.rstrip('\ufeff'))
LDN_geo_final = pd.concat([pd.to_numeric(LDN_geo_dec['Latitude'], errors='coerce'),LDN_geo_dec_L['Longitude'].astype(float),LDN_geo_DMS[['Latitude_DMS','Longitude_DMS']]],axis=1)

LDN_geo_final

,Latitude,Longitude,Latitude_DMS,Longitude_DMS
0,51.5607,0.1557,51°33′39″N 0°09′21″E﻿,﻿51.5607°N 0.1557°E﻿
1,51.6252,-0.1517,51°37′31″N 0°09′06″W﻿,﻿51.6252°N 0.1517°W﻿
2,51.4549,0.1505,51°27′18″N 0°09′02″E﻿,﻿51.4549°N 0.1505°E﻿
3,51.5588,-0.2817,51°33′32″N 0°16′54″W﻿,﻿51.5588°N 0.2817°W﻿
4,51.4039,0.0198,51°24′14″N 0°01′11″E﻿,﻿51.4039°N 0.0198°E﻿
5,51.5290,-0.1255,51°31′44″N 0°07′32″W﻿,﻿51.5290°N 0.1255°W﻿
6,51.3714,-0.0977,51°22′17″N 0°05′52″W﻿,﻿51.3714°N 0.0977°W﻿
7,51.5130,-0.3089,51°30′47″N 0°18′32″W﻿,﻿51.5130°N 0.3089°W﻿
8,51.6538,-0.0799,51°39′14″N 0°04′48″W﻿,﻿51.6538°N 0.0799°W﻿
9,51.4892,0.0648,51°29′21″N 0°03′53″E﻿,﻿51.4892°N 0.0648°E﻿


In [79]:
LDN_geo_final.dtypes

Latitude         float64
Longitude        float64
Latitude_DMS      object
Longitude_DMS     object
dtype: object

In [80]:
# add the new coordinates the the initial dataframe and clean up unwanted characters from the borough names
LDN_clean_final = pd.concat([LDN_clean.iloc[:,:-1],LDN_geo_final],axis=1)

LDN_clean_final

,Borough,Headquarters,Area (sq mi),Population (2013 est)[1],Political control,Latitude,Longitude,Latitude_DMS,Longitude_DMS
0,Barking and Dagenham [note 1],"Town Hall, 1 Town Square",13.93,"194,352",Labour,51.5607,0.1557,51°33′39″N 0°09′21″E﻿,﻿51.5607°N 0.1557°E﻿
1,Barnet,"North London Business Park, Oakleigh Road South",33.49,"369,088",Conservative,51.6252,-0.1517,51°37′31″N 0°09′06″W﻿,﻿51.6252°N 0.1517°W﻿
2,Bexley,"Civic Offices, 2 Watling Street",23.38,"236,687",Conservative,51.4549,0.1505,51°27′18″N 0°09′02″E﻿,﻿51.4549°N 0.1505°E﻿
3,Brent,"Brent Civic Centre, Engineers Way",16.70,"317,264",Labour,51.5588,-0.2817,51°33′32″N 0°16′54″W﻿,﻿51.5588°N 0.2817°W﻿
4,Bromley,"Civic Centre, Stockwell Close",57.97,"317,899",Conservative,51.4039,0.0198,51°24′14″N 0°01′11″E﻿,﻿51.4039°N 0.0198°E﻿
5,Camden,"Camden Town Hall, Judd Street",8.40,"229,719",Labour,51.5290,-0.1255,51°31′44″N 0°07′32″W﻿,﻿51.5290°N 0.1255°W﻿
6,Croydon,"Bernard Weatherill House, Mint Walk",33.41,"372,752",Labour,51.3714,-0.0977,51°22′17″N 0°05′52″W﻿,﻿51.3714°N 0.0977°W﻿
7,Ealing,"Perceval House, 14-16 Uxbridge Road",21.44,"342,494",Labour,51.5130,-0.3089,51°30′47″N 0°18′32″W﻿,﻿51.5130°N 0.3089°W﻿
8,Enfield,"Civic Centre, Silver Street",31.74,"320,524",Labour,51.6538,-0.0799,51°39′14″N 0°04′48″W﻿,﻿51.6538°N 0.0799°W﻿
9,Greenwich [note 2],"Woolwich Town Hall, Wellington Street",18.28,"264,008",Labour,51.4892,0.0648,51°29′21″N 0°03′53″E﻿,﻿51.4892°N 0.0648°E﻿


### Create a map of London with boroughs superimposed on top.

In [81]:
# create map of London using latitude and longitude values
address = 'London, UK'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_LDN = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(LDN_clean_final['Latitude'], LDN_clean_final['Longitude'], LDN_clean_final['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_LDN)  
    
map_LDN

/Users/alicechen/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


### Run analysis on all boroughs

##### Define Foursquare Credentials and Version

In [201]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


##### Run analysis for one borough as example

In [85]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [200]:
# create a function to extract and clean Foursquare venue data

def getNearbyVenues(names, latitudes, longitudes, radius=3000, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [140]:
LDN_clean_final.head()

,Borough,Headquarters,Area (sq mi),Population (2013 est)[1],Political control,Latitude,Longitude,Latitude_DMS,Longitude_DMS
0,Barking and Dagenham [note 1],"Town Hall, 1 Town Square",13.93,"194,352",Labour,51.5607,0.1557,51°33′39″N 0°09′21″E﻿,﻿51.5607°N 0.1557°E﻿
1,Barnet,"North London Business Park, Oakleigh Road South",33.49,"369,088",Conservative,51.6252,-0.1517,51°37′31″N 0°09′06″W﻿,﻿51.6252°N 0.1517°W﻿
2,Bexley,"Civic Offices, 2 Watling Street",23.38,"236,687",Conservative,51.4549,0.1505,51°27′18″N 0°09′02″E﻿,﻿51.4549°N 0.1505°E﻿
3,Brent,"Brent Civic Centre, Engineers Way",16.70,"317,264",Labour,51.5588,-0.2817,51°33′32″N 0°16′54″W﻿,﻿51.5588°N 0.2817°W﻿
4,Bromley,"Civic Centre, Stockwell Close",57.97,"317,899",Conservative,51.4039,0.0198,51°24′14″N 0°01′11″E﻿,﻿51.4039°N 0.0198°E﻿


In [141]:
# run analysis for all postcodes
LDN_venues = getNearbyVenues(names=LDN_clean_final['Borough'],
                                   latitudes=LDN_clean_final['Latitude'],
                                   longitudes=LDN_clean_final['Longitude']
                                  )
print(LDN_venues.shape)
LDN_venues.head()

(3018, 7)


,Borough,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barking and Dagenham [note 1],51.5607,0.1557,Central Park,51.559560,0.161981,Park
1,Barking and Dagenham [note 1],51.5607,0.1557,The Eva Hart (Wetherspoon),51.570460,0.130342,Pub
2,Barking and Dagenham [note 1],51.5607,0.1557,The Range,51.575550,0.180254,Furniture / Home Store
3,Barking and Dagenham [note 1],51.5607,0.1557,Costa Coffee,51.576890,0.179497,Coffee Shop
4,Barking and Dagenham [note 1],51.5607,0.1557,Harrow Lodge Park,51.555648,0.197926,Park


In [142]:
LDN_venues.groupby('Borough').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Barking and Dagenham [note 1],76,76,76,76,76,76
Barnet,98,98,98,98,98,98
Bexley,75,75,75,75,75,75
Brent,100,100,100,100,100,100
Bromley,68,68,68,68,68,68
Camden,100,100,100,100,100,100
City of London,100,100,100,100,100,100
Croydon,86,86,86,86,86,86
Ealing,100,100,100,100,100,100


In [143]:
print('There are {} uniques categories.'.format(len(LDN_venues['Venue Category'].unique())))

There are 261 uniques categories.


##### Analyse each borough

In [144]:
# one hot encoding
LDN_onehot = pd.get_dummies(LDN_venues[['Venue Category']], prefix="", prefix_sep="")

# add borough column back to dataframe
LDN_onehot['Borough'] = LDN_venues['Borough'] 

# move borough column to the first column
fixed_columns = [LDN_onehot.columns[-1]] + list(LDN_onehot.columns[:-1])
LDN_onehot = LDN_onehot[fixed_columns]

LDN_onehot.head()

,Borough,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Canal,Canal Lock,Caribbean Restaurant,Castle,Caucasian Restaurant,Cave,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hockey Field,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outlet Mall,Outlet Store,Paintball Field,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Reservoir,Restaurant,Road,Rock Club,Roof Deck,Rugby Pitch,Rugby Stadium,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Soccer Field,Soccer Stadium,Social Club,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Trail,Train Station,Tram Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,Barking and Dagenham [note 1],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [145]:
LDN_onehot.shape

(3018, 262)

In [146]:
# roup rows by borough and by taking the mean of the frequency of occurrence of each category

LDN_grouped = LDN_onehot.groupby('Borough').mean().reset_index()
LDN_grouped

,Borough,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Canal,Canal Lock,Caribbean Restaurant,Castle,Caucasian Restaurant,Cave,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hockey Field,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outlet Mall,Outlet Store,Paintball Field,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Reservoir,Restaurant,Road,Rock Club,Roof Deck,Rugby Pitch,Rugby Stadium,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Soccer Field,Soccer Stadium,Social Club,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Trail,Train Station,Tram Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,Barking and Dagenham [note 1],0.00,0.00,0.00,0.013158,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.013158,0.013158,0.00,0.00,0.00,0.00,0.000000,0.013158,0.00,0.00,0.00,0.013158,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.026316,0.00,0.026316,0.00,0.000000,0.000000,0.00,0.00,0.000000,

#### Group venues by type according to the Location Selection Criteria described above

In [160]:
LDN_res = LDN_grouped.filter(like='Restaurant', axis=1)
LDN_res.head()

,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Brazilian Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Halal Restaurant,Indian Chinese Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Okonomiyaki Restaurant,Pakistani Restaurant,Persian Restaurant,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,0.0,0.0,0.013158,0.0,0.0,0.000000,0.013158,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.013158,0.0,0.0,0.026316,0.000000,0.0,0.000000,0.0,0.0,0.013158,0.013158,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.026316,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.010204,0.010204,0.0,0.0,0.010204,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.020408,0.010204,0.0,0.040816,0.0,0.0,0.000000,0.061224,0.010204,0.0,0.0,0.00,0.010204,0.0,0.010204,0.010204,0.0,0.010204,0.0,0.0,0.0,0.0,0.010204,0.0,0.000000,0.0,0.0,0.0,0.0,0.010204,0.0,0.010204,0.040816,0.0,0.0,0.0
2,0.0,0.0,0.026667,0.0,0.0,0.000000,0.000000,0.0,0.0,0.013333,0.0,0.0,0.0,0.0,0.013333,0.0,0.0,0.026667,0.000000,0.0,0.013333,0.0,0.0,0.000000,0.040000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.013333,0.013333,0.0,0.000000,0.0,0.0,0.0,0.0,0.013333,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.020000,0.0,0.0,0.020000,0.010000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.010000,0.000000,0.0,0.000000,0.0,0.0,0.080000,0.020000,0.000000,0.0,0.0,0.01,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.040000,0.0,0.020000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.014706,0.0,0.0,0.014706,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.014706,0.0,0.0,0.014706,0.000000,0.0,0.000000,0.0,0.0,0.029412,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.014706,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.014706,0.0,0.0,0.0


In [158]:
LDN_bar = LDN_grouped.filter(like='Bar', axis=1)
LDN_bar = LDN_bar.drop('Salon / Barbershop', axis=1)
LDN_bar.head()

,Bar,Beer Bar,Cocktail Bar,Hookah Bar,Hotel Bar,Juice Bar,Sports Bar,Whisky Bar,Wine Bar
0,0.013158,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0
1,0.010204,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0
2,0.013333,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0
3,0.020000,0.0,0.0,0.01,0.01,0.0,0.01,0.0,0.0
4,0.044118,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0


In [159]:
LDN_pub = LDN_grouped.filter(like='Pub', axis=1)
LDN_pub = LDN_pub.drop('Public Art', axis=1)
LDN_pub.head()

,Irish Pub,Pub
0,0.000000,0.039474
1,0.000000,0.061224
2,0.000000,0.160000
3,0.000000,0.030000
4,0.014706,0.132353


In [162]:
LDN_hotel = LDN_grouped.filter(like='Hotel', axis=1)
LDN_hotel = LDN_hotel.drop('Hotel Bar', axis=1)
LDN_hotel.head()

,Hotel
0,0.026316
1,0.010204
2,0.013333
3,0.060000
4,0.014706


In [164]:
LDN_gym = LDN_grouped.filter(like='Gym', axis=1)
LDN_gym.head()

,Boxing Gym,Climbing Gym,Gym,Gym / Fitness Center,Gym Pool
0,0.0,0.0,0.013158,0.013158,0.00
1,0.0,0.0,0.000000,0.030612,0.00
2,0.0,0.0,0.013333,0.013333,0.00
3,0.0,0.0,0.010000,0.030000,0.01
4,0.0,0.0,0.000000,0.058824,0.00


In [165]:
# create dataframe with all the criteria

LDN_res_total = pd.DataFrame(LDN_grouped['Borough'])
LDN_res_total['Restaurants'] = LDN_res.sum(axis=1)
LDN_res_total['Chinese Restaurants'] = LDN_grouped['Chinese Restaurant'] + LDN_grouped['Dim Sum Restaurant'] + LDN_grouped['Dumpling Restaurant']
LDN_res_total['%Chinese Restaurants'] = LDN_res_total['Chinese Restaurants'] / LDN_res_total['Restaurants']
LDN_res_total['Bars/Pubs'] = LDN_pub.sum(axis=1) + LDN_bar.sum(axis=1)
LDN_res_total['Hotels'] = LDN_hotel
LDN_res_total['Gyms'] = LDN_gym.sum(axis=1)
LDN_res_total

,Borough,Restaurants,Chinese Restaurants,%Chinese Restaurants,Bars/Pubs,Hotels,Gyms
0,Barking and Dagenham [note 1],0.118421,0.000000,0.000000,0.052632,0.026316,0.026316
1,Barnet,0.285714,0.010204,0.035714,0.071429,0.010204,0.030612
2,Bexley,0.173333,0.013333,0.076923,0.173333,0.013333,0.026667
3,Brent,0.230000,0.000000,0.000000,0.080000,0.060000,0.050000
4,Bromley,0.117647,0.000000,0.000000,0.191176,0.014706,0.058824
5,Camden,0.170000,0.010000,0.058824,0.070000,0.060000,0.010000
6,City of London,0.120000,0.000000,0.000000,0.130000,0.130000,0.030000
7,Croydon,0.255814,0.011628,0.045455,0.069767,0.046512,0.011628
8,Ealing,0.170000,0.000000,0.000000,0.180000,0.050000,0.030000
9,Enfield,0.216216,0.000000,0.000000,0.162162,0.027027,0.040541


In [166]:
LDN_res_total.shape

(33, 7)

In [167]:
LDN_grouped.shape

(33, 262)

In [97]:
num_top_venues = 5

for borough in LDN_grouped['Borough']:
    print("----"+borough+"----")
    temp = LDN_grouped[LDN_grouped['Borough'] == borough].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barking and Dagenham [note 1]----
                  venue  freq
0              Bus Stop  0.12
1           Sports Club  0.06
2  Gym / Fitness Center  0.06
3         Grocery Store  0.06
4          Soccer Field  0.06


----Barnet----
              venue  freq
0              Park  0.43
1               Pub  0.29
2              Café  0.14
3          Bus Stop  0.14
4  Pedestrian Plaza  0.00


----Bexley----
                  venue  freq
0                   Pub  0.15
1           Supermarket  0.06
2           Coffee Shop  0.06
3  Fast Food Restaurant  0.06
4        Clothing Store  0.06


----Brent----
            venue  freq
0             Bar  0.08
1     Coffee Shop  0.06
2           Hotel  0.06
3  Sandwich Place  0.05
4  Clothing Store  0.05


----Bromley----
                  venue  freq
0                   Pub  0.11
1           Coffee Shop  0.11
2        Clothing Store  0.08
3  Gym / Fitness Center  0.05
4                  Park  0.03


----Camden----
         venue  freq
0         Café  

In [98]:
# function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [99]:
# create the new dataframe and display the top 10 venues for each borough
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Borough'] = LDN_grouped['Borough']

for ind in np.arange(LDN_grouped.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(LDN_grouped.iloc[ind, :], num_top_venues)

borough_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham [note 1],Bus Stop,Sports Club,Gym / Fitness Center,Construction & Landscaping,Chinese Restaurant,Park,Soccer Field,Bus Station,Supermarket,Gas Station
1,Barnet,Park,Pub,Café,Bus Stop,Fast Food Restaurant,Falafel Restaurant,Farm,Farmers Market,Filipino Restaurant,Food Stand
2,Bexley,Pub,Supermarket,Fast Food Restaurant,Coffee Shop,Clothing Store,Hotel,Chinese Restaurant,Furniture / Home Store,Pharmacy,Italian Restaurant
3,Brent,Bar,Coffee Shop,Hotel,Sandwich Place,Sporting Goods Shop,Clothing Store,Indian Restaurant,Supermarket,Restaurant,Italian Restaurant
4,Bromley,Pub,Coffee Shop,Clothing Store,Gym / Fitness Center,Indian Restaurant,Pizza Place,Park,Bar,Burger Joint,Café
5,Camden,Café,Hotel,Pub,Coffee Shop,Burger Joint,Breakfast Spot,Pizza Place,Plaza,Art Gallery,Restaurant
6,City of London,Coffee Shop,Italian Restaurant,Scenic Lookout,Gym / Fitness Center,Art Gallery,Restaurant,Pub,Hotel,Clothing Store,Park
7,Croydon,Coffee Shop,Pub,Clothing Store,Hotel,Café,Mediterranean Restaurant,Bookstore,Platform,Indian Restaurant,Tram Station
8,Ealing,Coffee Shop,Pub,Burger Joint,Pizza Place,Indian Restaurant,Italian Restaurant,Clothing Store,Hotel,Grocery Store,Café
9,Enfield,Pub,Coffee Shop,Clothing Store,Fish & Chips Shop,Supermarket,Indian Restaurant,Bookstore,Gift Shop,Italian Restaurant,Train Station


### Run Cluster Analysis

In [168]:
LDN_grouped_clustering = LDN_res_total.drop('Borough', 1)
LDN_grouped_clustering.head()

,Restaurants,Chinese Restaurants,%Chinese Restaurants,Bars/Pubs,Hotels,Gyms
0,0.118421,0.000000,0.000000,0.052632,0.026316,0.026316
1,0.285714,0.010204,0.035714,0.071429,0.010204,0.030612
2,0.173333,0.013333,0.076923,0.173333,0.013333,0.026667
3,0.230000,0.000000,0.000000,0.080000,0.060000,0.050000
4,0.117647,0.000000,0.000000,0.191176,0.014706,0.058824


In [187]:
# Run k-means to cluster the boroughs into 8 clusters.
# set number of clusters
kclusters = 8

LDN_grouped_clustering = LDN_res_total.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(LDN_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 2, 3, 7, 4, 5, 0, 7, 4, 1], dtype=int32)

In [188]:
len(kmeans.labels_)

33

In [189]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each
LDN_merged = LDN_res_total

# add clustering labels
LDN_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with merged_table to add latitude/longitude for each borough
LDN_merged = LDN_merged.join(LDN_clean_final.set_index('Borough'), on='Borough')

# drop rows with #NA
LDN_merged.dropna(axis=0)

LDN_merged.head() # check the last columns!

,Borough,Restaurants,Chinese Restaurants,%Chinese Restaurants,Bars/Pubs,Hotels,Gyms,Cluster Labels,Headquarters,Area (sq mi),Population (2013 est)[1],Political control,Latitude,Longitude,Latitude_DMS,Longitude_DMS
0,Barking and Dagenham [note 1],0.118421,0.000000,0.000000,0.052632,0.026316,0.026316,5,"Town Hall, 1 Town Square",13.93,"194,352",Labour,51.5607,0.1557,51°33′39″N 0°09′21″E﻿,﻿51.5607°N 0.1557°E﻿
1,Barnet,0.285714,0.010204,0.035714,0.071429,0.010204,0.030612,2,"North London Business Park, Oakleigh Road South",33.49,"369,088",Conservative,51.6252,-0.1517,51°37′31″N 0°09′06″W﻿,﻿51.6252°N 0.1517°W﻿
2,Bexley,0.173333,0.013333,0.076923,0.173333,0.013333,0.026667,3,"Civic Offices, 2 Watling Street",23.38,"236,687",Conservative,51.4549,0.1505,51°27′18″N 0°09′02″E﻿,﻿51.4549°N 0.1505°E﻿
3,Brent,0.230000,0.000000,0.000000,0.080000,0.060000,0.050000,7,"Brent Civic Centre, Engineers Way",16.70,"317,264",Labour,51.5588,-0.2817,51°33′32″N 0°16′54″W﻿,﻿51.5588°N 0.2817°W﻿
4,Bromley,0.117647,0.000000,0.000000,0.191176,0.014706,0.058824,4,"Civic Centre, Stockwell Close",57.97,"317,899",Conservative,51.4039,0.0198,51°24′14″N 0°01′11″E﻿,﻿51.4039°N 0.0198°E﻿


##### Visualize the resulting clusters

In [190]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(LDN_merged['Latitude'], LDN_merged['Longitude'], LDN_merged['Borough'], LDN_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### Examine Clusters

In [191]:
LDN_merged.loc[LDN_merged['Cluster Labels'] == 0]

,Borough,Restaurants,Chinese Restaurants,%Chinese Restaurants,Bars/Pubs,Hotels,Gyms,Cluster Labels,Headquarters,Area (sq mi),Population (2013 est)[1],Political control,Latitude,Longitude,Latitude_DMS,Longitude_DMS
6,City of London,0.12,0.0,0.0,0.13,0.13,0.03,0,Guildhall,1.12,"7,000",?,51.5155,-0.0922,51°30′56″N 0°05′32″W﻿,﻿51.5155°N 0.0922°W﻿
27,Southwark,0.18,0.0,0.0,0.11,0.08,0.04,0,160 Tooley Street,11.14,"298,464",Labour,51.5035,-0.0804,51°30′13″N 0°04′49″W﻿,﻿51.5035°N 0.0804°W﻿
32,Westminster,0.18,0.0,0.0,0.07,0.14,0.01,0,"Westminster City Hall, 64 Victoria Street",8.29,"226,841",Conservative,51.4973,-0.1372,51°29′50″N 0°08′14″W﻿,﻿51.4973°N 0.1372°W﻿


In [192]:
LDN_merged.loc[LDN_merged['Cluster Labels'] == 1]
# loads of restaurants, none Chinese, plenty of bars/pubs and few gyms, but not so many hotels
# given it's more of a residential area, we like it less
# Chinese food is unhealthy, so it's not recommended to have it often - thus the number of repeat customers is small
# tourists, on the other hand, tend to want to try new things, and areas with plenty of hotels guarantee a constant flow of tourists, which is what we like

,Borough,Restaurants,Chinese Restaurants,%Chinese Restaurants,Bars/Pubs,Hotels,Gyms,Cluster Labels,Headquarters,Area (sq mi),Population (2013 est)[1],Political control,Latitude,Longitude,Latitude_DMS,Longitude_DMS
9,Enfield,0.216216,0.0,0.0,0.162162,0.027027,0.040541,1,"Civic Centre, Silver Street",31.74,"320,524",Labour,51.6538,-0.0799,51°39′14″N 0°04′48″W﻿,﻿51.6538°N 0.0799°W﻿
12,Hammersmith and Fulham [note 4],0.250000,0.0,0.0,0.150000,0.020000,0.000000,1,"Town Hall, King Street",6.33,"178,685",Labour,51.4927,-0.2339,51°29′34″N 0°14′02″W﻿,﻿51.4927°N 0.2339°W﻿
18,Islington,0.230000,0.0,0.0,0.150000,0.010000,0.010000,1,"Municipal Offices, 222 Upper Street",5.74,"215,667",Labour,51.5416,-0.1022,51°32′30″N 0°06′08″W﻿,﻿51.5416°N 0.1022°W﻿
20,Kingston upon Thames,0.240000,0.0,0.0,0.170000,0.030000,0.030000,1,"Guildhall, High Street",14.38,"166,793",Liberal Democrat,51.4085,-0.3064,51°24′31″N 0°18′23″W﻿,﻿51.4085°N 0.3064°W﻿
30,Waltham Forest,0.210000,0.0,0.0,0.180000,0.020000,0.040000,1,"Waltham Forest Town Hall, Forest Road",14.99,"265,797",Labour,51.5908,-0.0134,51°35′27″N 0°00′48″W﻿,﻿51.5908°N 0.0134°W﻿


In [193]:
LDN_merged.loc[LDN_merged['Cluster Labels'] == 2]
#plenty of restaurants, quite a few Chinese, some bars/pubs, few gyms but very few hotels
#there are already Chinese restaurants in the area, and it does't appear too touristy (judging by the small number of hotels)


,Borough,Restaurants,Chinese Restaurants,%Chinese Restaurants,Bars/Pubs,Hotels,Gyms,Cluster Labels,Headquarters,Area (sq mi),Population (2013 est)[1],Political control,Latitude,Longitude,Latitude_DMS,Longitude_DMS
1,Barnet,0.285714,0.010204,0.035714,0.071429,0.010204,0.030612,2,"North London Business Park, Oakleigh Road South",33.49,"369,088",Conservative,51.6252,-0.1517,51°37′31″N 0°09′06″W﻿,﻿51.6252°N 0.1517°W﻿
13,Haringey,0.300000,0.010000,0.033333,0.080000,0.000000,0.020000,2,"Civic Centre, High Road",11.42,"263,386",Labour,51.6000,-0.1119,51°36′00″N 0°06′43″W﻿,﻿51.6000°N 0.1119°W﻿
14,Harrow,0.271605,0.024691,0.090909,0.123457,0.012346,0.061728,2,"Civic Centre, Station Road",19.49,"243,372",Labour,51.5898,-0.3346,51°35′23″N 0°20′05″W﻿,﻿51.5898°N 0.3346°W﻿
21,Lambeth,0.270000,0.020000,0.074074,0.140000,0.000000,0.010000,2,"Lambeth Town Hall, Brixton Hill",10.36,"314,242",Labour,51.4607,-0.1163,51°27′39″N 0°06′59″W﻿,﻿51.4607°N 0.1163°W﻿
23,Merton,0.317647,0.011765,0.037037,0.105882,0.011765,0.047059,2,"Civic Centre, London Road",14.52,"203,223",Labour,51.4014,-0.1958,51°24′05″N 0°11′45″W﻿,﻿51.4014°N 0.1958°W﻿


In [194]:
LDN_merged.loc[LDN_merged['Cluster Labels'] == 3]
#similar to cluster 2

,Borough,Restaurants,Chinese Restaurants,%Chinese Restaurants,Bars/Pubs,Hotels,Gyms,Cluster Labels,Headquarters,Area (sq mi),Population (2013 est)[1],Political control,Latitude,Longitude,Latitude_DMS,Longitude_DMS
2,Bexley,0.173333,0.013333,0.076923,0.173333,0.013333,0.026667,3,"Civic Offices, 2 Watling Street",23.38,"236,687",Conservative,51.4549,0.1505,51°27′18″N 0°09′02″E﻿,﻿51.4549°N 0.1505°E﻿
11,Hackney,0.200000,0.010000,0.050000,0.130000,0.010000,0.010000,3,"Hackney Town Hall, Mare Street",7.36,"257,379",Labour,51.5450,-0.0553,51°32′42″N 0°03′19″W﻿,﻿51.5450°N 0.0553°W﻿
26,Richmond upon Thames,0.210000,0.010000,0.047619,0.190000,0.040000,0.000000,3,"Civic Centre, 44 York Street",22.17,"191,365",Liberal Democrat,51.4479,-0.3260,51°26′52″N 0°19′34″W﻿,﻿51.4479°N 0.3260°W﻿
28,Sutton,0.191176,0.014706,0.076923,0.147059,0.029412,0.044118,3,"Civic Offices, St Nicholas Way",16.93,"195,914",Liberal Democrat,51.3618,-0.1945,51°21′42″N 0°11′40″W﻿,﻿51.3618°N 0.1945°W﻿


In [195]:
LDN_merged.loc[LDN_merged['Cluster Labels'] == 4]
#similar to cluster 1

,Borough,Restaurants,Chinese Restaurants,%Chinese Restaurants,Bars/Pubs,Hotels,Gyms,Cluster Labels,Headquarters,Area (sq mi),Population (2013 est)[1],Political control,Latitude,Longitude,Latitude_DMS,Longitude_DMS
4,Bromley,0.117647,0.0,0.0,0.191176,0.014706,0.058824,4,"Civic Centre, Stockwell Close",57.97,"317,899",Conservative,51.4039,0.0198,51°24′14″N 0°01′11″E﻿,﻿51.4039°N 0.0198°E﻿
8,Ealing,0.170000,0.0,0.0,0.180000,0.050000,0.030000,4,"Perceval House, 14-16 Uxbridge Road",21.44,"342,494",Labour,51.5130,-0.3089,51°30′47″N 0°18′32″W﻿,﻿51.5130°N 0.3089°W﻿
22,Lewisham,0.150000,0.0,0.0,0.200000,0.000000,0.020000,4,"Town Hall, 1 Catford Road",13.57,"286,180",Labour,51.4452,-0.0209,51°26′43″N 0°01′15″W﻿,﻿51.4452°N 0.0209°W﻿
31,Wandsworth,0.120000,0.0,0.0,0.160000,0.010000,0.030000,4,"The Town Hall, Wandsworth High Street",13.23,"310,516",Conservative,51.4567,-0.1910,51°27′24″N 0°11′28″W﻿,﻿51.4567°N 0.1910°W﻿


In [196]:
LDN_merged.loc[LDN_merged['Cluster Labels'] == 5]
#similar to cluster 1

,Borough,Restaurants,Chinese Restaurants,%Chinese Restaurants,Bars/Pubs,Hotels,Gyms,Cluster Labels,Headquarters,Area (sq mi),Population (2013 est)[1],Political control,Latitude,Longitude,Latitude_DMS,Longitude_DMS
0,Barking and Dagenham [note 1],0.118421,0.00,0.000000,0.052632,0.026316,0.026316,5,"Town Hall, 1 Town Square",13.93,"194,352",Labour,51.5607,0.1557,51°33′39″N 0°09′21″E﻿,﻿51.5607°N 0.1557°E﻿
5,Camden,0.170000,0.01,0.058824,0.070000,0.060000,0.010000,5,"Camden Town Hall, Judd Street",8.40,"229,719",Labour,51.5290,-0.1255,51°31′44″N 0°07′32″W﻿,﻿51.5290°N 0.1255°W﻿
15,Havering,0.151515,0.00,0.000000,0.075758,0.030303,0.045455,5,"Town Hall, Main Road",43.35,"242,080",Conservative (council NOC),51.5812,0.1837,51°34′52″N 0°11′01″E﻿,﻿51.5812°N 0.1837°E﻿
29,Tower Hamlets,0.150000,0.00,0.000000,0.090000,0.050000,0.060000,5,"Town Hall, Mulberry Place, 5 Clove Crescent",7.63,"272,890",Labour,51.5099,-0.0059,51°30′36″N 0°00′21″W﻿,﻿51.5099°N 0.0059°W﻿


In [198]:
LDN_merged.loc[LDN_merged['Cluster Labels'] == 6]
#similar to cluster 2
# high % of Chinese rest, more gyms, fewer bars/pubs

,Borough,Restaurants,Chinese Restaurants,%Chinese Restaurants,Bars/Pubs,Hotels,Gyms,Cluster Labels,Headquarters,Area (sq mi),Population (2013 est)[1],Political control,Latitude,Longitude,Latitude_DMS,Longitude_DMS
10,Greenwich [note 2],0.11,0.01,0.090909,0.10,0.11,0.05,6,"Woolwich Town Hall, Wellington Street",18.28,"264,008",Labour,51.4892,0.0648,51°29′21″N 0°03′53″E﻿,﻿51.4892°N 0.0648°E﻿
17,Hounslow,0.18,0.01,0.055556,0.08,0.07,0.06,6,"Civic Centre, Lampton Road",21.61,"262,407",Labour,51.4746,-0.3680,51°28′29″N 0°22′05″W﻿,﻿51.4746°N 0.3680°W﻿
24,Newham,0.13,0.01,0.076923,0.13,0.10,0.06,6,"Newham Dockside, 1000 Dockside Road",13.98,"318,227",Labour,51.5077,0.0469,51°30′28″N 0°02′49″E﻿,﻿51.5077°N 0.0469°E﻿


In [199]:
LDN_merged.loc[LDN_merged['Cluster Labels'] == 7]
#similar to cluster 2

,Borough,Restaurants,Chinese Restaurants,%Chinese Restaurants,Bars/Pubs,Hotels,Gyms,Cluster Labels,Headquarters,Area (sq mi),Population (2013 est)[1],Political control,Latitude,Longitude,Latitude_DMS,Longitude_DMS
3,Brent,0.230000,0.000000,0.000000,0.080000,0.060000,0.050000,7,"Brent Civic Centre, Engineers Way",16.70,"317,264",Labour,51.5588,-0.2817,51°33′32″N 0°16′54″W﻿,﻿51.5588°N 0.2817°W﻿
7,Croydon,0.255814,0.011628,0.045455,0.069767,0.046512,0.011628,7,"Bernard Weatherill House, Mint Walk",33.41,"372,752",Labour,51.3714,-0.0977,51°22′17″N 0°05′52″W﻿,﻿51.3714°N 0.0977°W﻿
16,Hillingdon,0.236842,0.013158,0.055556,0.105263,0.026316,0.039474,7,"Civic Centre, High Street",44.67,"286,806",Conservative,51.5441,-0.4760,51°32′39″N 0°28′34″W﻿,﻿51.5441°N 0.4760°W﻿
19,Kensington and Chelsea,0.290000,0.010000,0.034483,0.060000,0.060000,0.010000,7,"The Town Hall, Hornton Street",4.68,"155,594",Conservative,51.5020,-0.1947,51°30′07″N 0°11′41″W﻿,﻿51.5020°N 0.1947°W﻿
25,Redbridge,0.215385,0.015385,0.071429,0.076923,0.030769,0.046154,7,"Town Hall, 128-142 High Road",21.78,"288,272",Labour,51.5590,0.0741,51°33′32″N 0°04′27″E﻿,﻿51.5590°N 0.0741°E﻿


# Conclusion

### We notice that, according to the location selection criteria, cluster 0 has the most desirable attributes:
#### - relatively high number of restaurants
#### - no Chinese restaurants
#### - plenty of bars/pubs
#### - plenty of hotels
#### - few gyms

### Furthermore, out of Cluster 0, we prefer Southwark as our location for the new Chinese restaurant, because it is more densely populate than the other two